In [1]:
import keras
import numpy as np
import pandas as pd
import pickle
import os
from statistics import mode
from collections import Counter

from keras_vggface.vggface import VGGFace
from keras_vggface import utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def mean_std(feature):
    mean, std = np.mean(feature, axis=0), np.std(feature, axis=0)
    return np.hstack((mean, std))

def change_labels(labels):
    counter = Counter(labels)
    if len(counter) > 1:
        try:
            if int(mode(labels)) == -1:
                curr_mode = int(sorted(counter, key=counter.get, reverse=True)[1])
            else:
                curr_mode = int(mode(labels))
        except:
            if int(sorted(counter, key=counter.get, reverse=True)[0]) == -1:
                curr_mode = int(sorted(counter, key=counter.get, reverse=True)[1])
            else:
                curr_mode = int(sorted(counter, key=counter.get, reverse=True)[0])
    else:
        curr_mode = int(mode(labels))
    return curr_mode

def new_feature_labels_name(name_folder_images, curr_index_need, curr_window, curr_feature, curr_labels, step):
    new_name = []
    new_index = []
    new_feature = []
    new_labels = []
    for i in range(0, len(curr_labels), round(curr_window/step)):
        new_name.append(name_folder_images)
        if i + curr_window > len(curr_labels):
            new_index.append(curr_index_need[i:len(curr_labels)])
            new_feature.append(mean_std(curr_feature[i:len(curr_labels)]))
            new_labels.append(change_labels(curr_labels[i:len(curr_labels)]))
        else:
            new_index.append(curr_index_need[i:i+curr_window])
            new_feature.append(mean_std(curr_feature[i:i+curr_window]))
            new_labels.append(change_labels(curr_labels[i:i+curr_window]))
    return new_name, new_index, new_feature, new_labels

def window_data(unique_name_video, df, feature, fps_vadeo, step):
    new_name_all = []
    new_index_all = []
    new_feature_all = []
    new_labels_all = []
    for name_folder_images in unique_name_video:
        curr_df = df.loc()[df.name_folder==name_folder_images].copy()
        curr_index_need = curr_df.index.tolist()
        if len(name_folder_images.split('_')) == 2 and len(name_folder_images.split('_')[1]) > 2:
            curr_name = name_folder_images.split('_')[0]
        else:
            curr_name = name_folder_images
        curr_window = int(fps_vadeo[fps_vadeo.name==curr_name].window.values)
        curr_feature = feature[min(curr_index_need):max(curr_index_need)+1]
        curr_labels = curr_df.emotion.tolist()
        new_name, new_index, new_feature, new_labels = new_feature_labels_name(name_folder_images, curr_index_need,curr_window, curr_feature, curr_labels, step)
        new_name_all += new_name
        new_index_all += new_index
        new_feature_all += new_feature
        new_labels_all += new_labels
        
    return new_name_all, new_index_all, new_feature_all, new_labels_all

def create_dict(name, index, feature, emotion):
    dict_ = {}
    dict_['name'] = name
    dict_['index'] = index
    dict_['feature'] = feature
    dict_['emotion'] = emotion
    return dict_

def data_minus_one(name,index, feature, emotion):
    # applies only to training set
    name_new = []
    index_new = []
    feature_new = []
    emotion_new = []

    for i in range(len(emotion)):
        if emotion[i] != -1:
            name_new.append(name[i])
            index_new.append(index[i])
            feature_new.append(feature[i])
            emotion_new.append(emotion[i])
    return name_new, index_new, feature_new, emotion_new

In [3]:
# preparing data for a end-to-end model
images = 'C:/Users/ryumi/Desktop/aff_wild/images_autors/'
filename = 'path_images_train.csv' # or path_images_valid or path_images_test
df_train = pd.read_csv('data_sequence_svm/' + filename)

In [4]:
datagen_valid = ImageDataGenerator(preprocessing_function=utils.preprocess_input)

size = (224,224)
bs = 32

train_generator = datagen_valid.flow_from_dataframe(dataframe=df_train,
                                                     directory=images,
                                                     x_col='path_images',
                                                     y_col='emotion',
                                                     target_size=size,
                                                     batch_size=bs,
                                                     class_mode=None,
                                                     shuffle=False,
                                                     )

In [5]:
path_model_img = 'C:/Users/ryumi/Desktop/code/models/resnet50/'

resnet50 = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
x = keras.layers.Dense(units = 1024,activation = 'relu', name='dense')(resnet50.output)
x = keras.layers.Dropout(0.5)(x) 
x = keras.layers.Dense(7, activation = 'softmax')(x)         
model_resnet50 = keras.models.Model(resnet50.input, x)
model_resnet50.load_weights(path_model_img + "/weights.h5")

model_loaded = keras.models.Model(inputs=model_resnet50.input, outputs=[model_resnet50.get_layer('dense').output])

In [6]:
feature_train = model_loaded.predict(train_generator, verbose=1)

In [7]:
# formation of feature for SVM
path_fps_vadeo = 'files_needed_to_get_final_probabilities/videos_frame_rate.txt'
fps_vadeo = pd.read_csv(path_fps_vadeo, dtype={'filename': str})

In [8]:
names_ids = fps_vadeo.filename.str.split('.')
res = names_ids.str[0]
fps_vadeo['name'] = res
window_video = [round(i*30/7.5) for i in fps_vadeo.frame_rate.tolist()] # window 4 seconds
fps_vadeo['window'] = window_video

In [9]:
unique_name_video_train = list(Counter(df_train.name_folder.tolist()).keys())

step = 2 # if the window is 30 frames, then the overlap is 50% (30/2)

name_train_new, index_train_new, feature_train_new, emotion_train_new = window_data(unique_name_video_train, df_train, feature_train, fps_vadeo, step)
name_train_minus_one, index_train_minus_one, feature_train_minus_one, emotion_train_minus_one = data_minus_one(name_train_new, index_train_new, feature_train_new, emotion_train_new)

In [10]:
# formation of dictionaries for subsequent training and predictions
dict_train = create_dict(name_train_new, index_train_new, feature_train_new, emotion_train_new)
dict_train_minus = create_dict(name_train_minus_one, index_train_minus_one, feature_train_minus_one, emotion_train_minus_one)

path = 'data_sequence_svm/'

with open(path+'dict_train_{}.pickle'.format('window_4_s'), 'wb') as f:
    pickle.dump(dict_train, f)
with open(path+'dict_train_minus_one_{}.pickle'.format('window_4_s'), 'wb') as f:
    pickle.dump(dict_train_minus, f)